In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, UpSampling2D
import numpy as np
import matplotlib.pyplot as plt
import urllib
from google.colab import files

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
(x_train,_),(x_test,_)= cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

170500096/170498071 [==============================] - 57s 0us/step


In [3]:
x_train.shape

(50000, 32, 32, 3)

In [0]:
x_train = x_train.reshape(x_train.shape[0],32,32,3)
x_test = x_test.reshape(x_test.shape[0],32,32,3)

In [0]:
import cv2

In [0]:
x_train_reduced = []
for i in range(len(x_train)):
  x_train_reduced.append(cv2.resize(x_train[i],dsize=(16,16),interpolation=cv2.INTER_CUBIC))
x_train_reduced = np.array(x_train_reduced)

In [0]:
x_test_reduced = []
for i in range(len(x_test)):
  x_test_reduced.append(cv2.resize(x_test[i],dsize=(16,16),interpolation=cv2.INTER_CUBIC))
x_test_reduced = np.array(x_test_reduced)

In [0]:
model = Sequential()

model.add(Conv2D(4,(4,4),activation = 'relu',padding='same',input_shape=(None,None,3)))
model.add(Conv2D(5,(3,3),activation='relu',padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(3,(3,3),activation = 'sigmoid',padding='same'))


In [0]:
model.compile(optimizer='adadelta',loss='binary_crossentropy')

In [0]:
history = model.fit(x_train_reduced,x_train,epochs=100,batch_size=128,shuffle=True,validation_data=(x_test_reduced,x_test),)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 13s 261us/step - loss: 0.6259 - val_loss: 0.5942
Epoch 2/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.5804 - val_loss: 0.5696
Epoch 3/100
50000/50000 [==============================] - 8s 169us/step - loss: 0.5691 - val_loss: 0.5670
Epoch 4/100
50000/50000 [==============================] - 8s 166us/step - loss: 0.5651 - val_loss: 0.5644
Epoch 5/100
50000/50000 [==============================] - 8s 165us/step - loss: 0.5634 - val_loss: 0.5631
Epoch 6/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.5624 - val_loss: 0.5632
Epoch 7/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.5617 - val_loss: 0.5624
Epoch 8/100
50000/50000 [==============================] - 8s 167us/step - loss: 0.5613 - val_loss: 0.5623
Epoch 9/100
50000/50000 [==============================] - 8s 164us/step - loss: 0.5611 - val

In [0]:
predictions = model.predict(x_test_reduced)%1

In [0]:
plt.plot(history.history['val_loss'])

In [0]:
i=np.random.randint(x_test.shape[0])
f,axarr = plt.subplots(1,4)
axarr[0].imshow(x_test_reduced[i])
axarr[0].set_title = 'Downscaled'
axarr[1].imshow(predictions[i])
axarr[1].set_title = 'Predicted'
axarr[2].imshow(x_test[i])
axarr[2].set_title='Actual'
axarr[3].imshow(cv2.resize(x_test[i],dsize=(32,32),interpolation=cv2.INTER_CUBIC))


In [0]:
def enhance(url, smooth = True,overshoot_val = 1,dpi=80):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  img = np.flip(image,axis=-1)
  h = img.shape[0]*2
  w = img.shape[1]*2
  
  img = img/255
  
  prediction = model.predict(np.array([img]))
  for _ in range(overshoot_val):
    prediction = model.predict(prediction)
    
  
  prediction = np.clip(prediction,0,1)
  im_data = prediction[0]
  
  if smooth:
    im_data =  cv2.GaussianBlur(im_data,(15,15),0)
    im_data = cv2.resize(im_data,dsize=(w,h),interpolation=cv2.INTER_AREA)
  
  height, width, depth = im_data.shape
  figsize = width / float(dpi), height / float(dpi)
  fig = plt.figure(figsize=figsize)
  ax = fig.add_axes([0, 0, 1, 1])
  ax.axis('off')
  ax.imshow(im_data, cmap='gray')
  
  plt.savefig('foo.jpg')
  files.download('foo.jpg')
  
  
